# TRABAJO FINAL PROCESAMIENTO DEL LENGUAJE NATURAL 
ALUMNOS : 

- ALEJANDRO MADRID GALARZA
- ANTONIO JOSÉ LÓPEZ MARTÍNEZ


### MOTIVACIÓN DEL TRABAJO
Vamos a realizar una aplicacion 'python' en 'jupyter-lab' para la asignatura de Procesamiento del Lenguaje Natural en la que trataremos, con los conocimientos adquiridos en la asignatura así como todo lo que sea necesario para la resolución del mismo, una aplicación que clasifique un conjunto de más de 3500 'tweets' en un clasificador de 11 clases que corresponden a 11 emociones distintas sobre las que clasificaremos.

In [101]:
# IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import spacy
#
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.svm import SVC
#
from unidecode import unidecode
from spacy.lang.es.stop_words import STOP_WORDS
#

In [102]:
# CARGAMOS LOS DATOS
train_data = pd.read_csv('Data/Train/sem_eval_train_es.csv')
test_data = pd.read_csv('Data/Test/sem_eval_test_grupo_10.csv')

In [103]:
train_data.shape, test_data.shape

((3561, 13), (679, 2))

In [104]:
# Vamos a eliminar los id's de ambos csv
train_data = train_data.drop(['ID'], axis=1)
test_data = test_data.drop(['ID'], axis=1)
print(train_data.shape, test_data.shape)
print(train_data.head())
print(test_data.head())

(3561, 12) (679, 1)
                                               Tweet  anger  anticipation  \
0  @aliciaenp Ajajjaa somos del clan twitteras pe...  False         False   
1  @AwadaNai la mala suerte del gato fichame la c...  False         False   
2  @audiomano A mí tampoco me agrado mucho eso. E...   True         False   
3  Para llevar a los bebes de un lugar a otro deb...  False         False   
4  @DalasReview me encanta la terrible hipocresia...   True         False   

   disgust   fear    joy   love  optimism  pessimism  sadness  surprise  trust  
0    False  False   True  False     False      False    False     False  False  
1    False   True  False  False     False       True    False     False  False  
2    False  False  False  False     False      False    False     False  False  
3    False  False   True  False     False      False    False     False  False  
4     True  False  False  False     False      False    False     False  False  
                               

Ahora que hemos eliminado la columna de ID's de los conjuntos de entrenamiento y testeo tenemos que separar nuestros tweets de testeo que, como podemos observar tiene 12 dimensiones (11 de las diferentes emociones + 1 para los tweets). 
Los vamos a separar como: 
- $X=contenido$ $tweets$
- $Y=categoría$ $tweet$

In [105]:
X = train_data['Tweet']
y = train_data.drop(['Tweet'], axis=1)
print(X.shape, y.shape, '\n')
print(X.head(), '\n')
print(y.head())

(3561,) (3561, 11) 

0    @aliciaenp Ajajjaa somos del clan twitteras pe...
1    @AwadaNai la mala suerte del gato fichame la c...
2    @audiomano A mí tampoco me agrado mucho eso. E...
3    Para llevar a los bebes de un lugar a otro deb...
4    @DalasReview me encanta la terrible hipocresia...
Name: Tweet, dtype: object 

   anger  anticipation  disgust   fear    joy   love  optimism  pessimism  \
0  False         False    False  False   True  False     False      False   
1  False         False    False   True  False  False     False       True   
2   True         False    False  False  False  False     False      False   
3  False         False    False  False   True  False     False      False   
4   True         False     True  False  False  False     False      False   

   sadness  surprise  trust  
0    False     False  False  
1    False     False  False  
2    False     False  False  
3    False     False  False  
4    False     False  False  


### Preprocesado de datos
Ahora solo vamos a trabajar con el conjunto de entrenamiento ya que tenemos que masticar los datos y pasárselos al modelo que posteriormente entrenamremos

In [106]:
# Comenzamos el preprocesado cargando el modelo a ejecutar
nlp=spacy.load('Data/Model/es_core_news_sm-3.7.0-py3-none-any/es_core_news_sm/es_core_news_sm-3.7.0')

In [107]:
# En esta celda vamos a definir las funciones para la limpieza del texto
def limpiar_texto(tweet):
    tweet = unidecode(tweet)                                                            # Elimina los acentos
    tweet = re.sub(r'@[A-Za-z0-9_]+', '', tweet)                                        # Eliminamos mencinoes
    doc = nlp(tweet)                                                                    # Tokenizamos el tweet
    tokens = [token.text for token in doc if not token.is_stop and len(token.text) > 1] # Eliminamos stop words y palabras de 1 letra
    return " ".join(tokens)                                                             # Unimos los tokens, único string
              

In [108]:
# Limpiamos nuestros tweets
X = X.apply(limpiar_texto)
print(X.head())

0    Ajajjaa clan twitteras perdidas pa eventos imp...
1               mala suerte gato fichame cara help pls
2          agrado Especialmente tratarse justificacion
3    bebes lugar debemos cantarles canciones ... Qu...
4    encanta terrible hipocresia doble moral gente ...
Name: Tweet, dtype: object


In [109]:
# Ahora que tenemos los datos limpios vamos a separar en entrenamiento y testeo para el modelo 
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3561,) (3561, 11)
(2848,) (713,) (2848, 11) (713, 11)


In [110]:
# Creamos la representación vecrotial de nuestros datos por el método TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
# Además vamos a transformar nuestros y en 1-D para los modelos
y_train = y_train.idxmax(axis=1)
y_test = y_test.idxmax(axis=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2848, 6899) (713, 6899) (2848,) (713,)


Una vez creada la BOW, en nuestro caso por el método TF-IDF tenemos que crear el modelo que usaremos para la predicció y entrenarlo con nuestros datos de entrenamiento. 
### Modelo
En nuestro caso vamos a realizar un pequeño estudio sobre 3 modelos para saber cuál es mejor para nuestro Dataset y luego decidiremos el mejor y lo usaremospara la predicción. 
Los modelos a tratar va a ser:
- Multinomial Naive Bayes
- SVM
- Transformer --> BERT

#### Naive Bayes

In [111]:
# Creamos el modelo Naive Bayes
nbModelo = MultinomialNB()
nbModelo.fit(X_train, y_train)
nbModelo.predict(X_test, y_test)

TypeError: _BaseNB.predict() takes 2 positional arguments but 3 were given